<a href="https://colab.research.google.com/github/E-S-P-I-A/Git/blob/main/PW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import math

# Вспомогательные функции

def is_prime(n, k=5):
    """Проверка числа на простоту с использованием теста Ферма"""
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0:
        return False

    # Представим n-1 в виде 2^r * d
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2

    # Выполним k тестов
    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

def generate_prime(bits):
    """Генерация случайного простого числа заданной битовой длины"""
    while True:
        # Генерируем случайное число нужной битовой длины
        num = random.getrandbits(bits)
        # Убедимся, что число нечетное
        num |= 1
        # Проверим на простоту
        if is_prime(num):
            return num

def extended_gcd(a, b):
    """Расширенный алгоритм Евклида для нахождения НОД и коэффициентов Безу"""
    if a == 0:
        return b, 0, 1
    else:
        gcd, x, y = extended_gcd(b % a, a)
        return gcd, y - (b // a) * x, x

def mod_inverse(e, phi):
    """Нахождение мультипликативного обратного по модулю"""
    gcd, x, y = extended_gcd(e, phi)
    if gcd != 1:
        raise ValueError("Мультипликативного обратного не существует")
    else:
        return x % phi

# Основные функции RSA

def generate_keypair(bits=1024):
    """Генерация ключевой пары RSA"""
    # Генерируем два простых числа p и q
    p = generate_prime(bits // 2)
    q = generate_prime(bits // 2)

    # Вычисляем модуль n и функцию Эйлера phi(n)
    n = p * q
    phi = (p - 1) * (q - 1)

    # Выбираем открытую экспоненту e
    e = 65537  # Стандартное значение, обычно используемое в RSA

    # Вычисляем закрытую экспоненту d
    d = mod_inverse(e, phi)

    return ((e, n), (d, n))

def encrypt(message, public_key):
    """Зашифрование сообщения с использованием открытого ключа"""
    e, n = public_key

    # Преобразуем сообщение в массив чисел
    message_nums = [ord(char) for char in message]

    # Зашифровываем каждое число
    cipher = [pow(m, e, n) for m in message_nums]

    return cipher

def decrypt(cipher, private_key):
    """Расшифрование сообщения с использованием закрытого ключа"""
    d, n = private_key

    # Расшифровываем каждое число
    message_nums = [pow(c, d, n) for c in cipher]

    # Преобразуем числа обратно в символы
    message = ''.join([chr(m) for m in message_nums])

    return message

# Функции для работы с файлами

def save_key_to_file(key, filename):
    """Сохранение ключа в файл"""
    with open(filename, 'w') as f:
        f.write(f"{key[0]},{key[1]}")

def load_key_from_file(filename):
    """Загрузка ключа из файла"""
    with open(filename, 'r') as f:
        content = f.read().strip()
        parts = content.split(',')
        return (int(parts[0]), int(parts[1]))

def encrypt_file(input_filename, output_filename, public_key):
    """Зашифрование содержимого файла"""
    with open(input_filename, 'r') as f:
        message = f.read()

    cipher = encrypt(message, public_key)

    with open(output_filename, 'w') as f:
        f.write(','.join([str(c) for c in cipher]))

def decrypt_file(input_filename, output_filename, private_key):
    """Расшифрование содержимого файла"""
    with open(input_filename, 'r') as f:
        content = f.read().strip()
        cipher = [int(c) for c in content.split(',')]

    message = decrypt(cipher, private_key)

    with open(output_filename, 'w') as f:
        f.write(message)

# Пример использования
if __name__ == "__main__":
    # Генерация ключевой пары
    public_key, private_key = generate_keypair(bits=2048)
    print(f"Открытый ключ (e, n): {public_key}")
    print(f"Закрытый ключ (d, n): {private_key}")

    # Пример шифрования и расшифрования
    message = "Hello, RSA!"
    cipher = encrypt(message, public_key)
    decrypted = decrypt(cipher, private_key)

    print(f"Исходное сообщение: {message}")
    print(f"Зашифрованное сообщение: {cipher}")
    print(f"Расшифрованное сообщение: {decrypted}")


In [ ]:
# Задаем фиксированные значения для сравнения с "ручным" расчетом
p = 11
q = 13
n = p * q  # n = 143
phi = (p - 1) * (q - 1)  # phi = 120
e = 7  # Открытая экспонента
d = mod_inverse(e, phi)  # d = 103

public_key = (e, n)
private_key = (d, n)

print(f"Открытый ключ (e, n): {public_key}")
print(f"Закрытый ключ (d, n): {private_key}")

message = "9"
cipher = encrypt(message, public_key)
decrypted = decrypt(cipher, private_key)

print(f"Исходное сообщение: {message}")
print(f"Зашифрованное сообщение: {cipher}")
print(f"Расшифрованное сообщение: {decrypted}")


In [ ]:
def continued_fraction_expansion(n, d):
    """Разложение числа n/d в непрерывную дробь"""
    result = []
    while d:
        q = n // d
        result.append(q)
        n, d = d, n - q * d
    return result

def convergents(e, n):
    """Нахождение подходящих дробей для e/n"""
    cfexp = continued_fraction_expansion(e, n)

    p = [0, 1]
    q = [1, 0]

    for i in range(len(cfexp)):
        p.append(cfexp[i] * p[i+1] + p[i])
        q.append(cfexp[i] * q[i+1] + q[i])

    return list(zip(p[2:], q[2:]))

def attack_wiener(e, n):
    """Атака Винера на RSA с малой закрытой экспонентой"""
    conv = convergents(e, n)

    for k, d in conv:
        # Проверяем, является ли d потенциальным закрытым ключом
        if k == 0:
            continue

        # ed - 1 должно делиться на phi(n)
        if (e * d - 1) % k == 0:
            phi = (e * d - 1) // k
            # Решаем квадратное уравнение x^2 - (n - phi + 1)x + n = 0
            a = 1
            b = -(n - phi + 1)
            c = n

            # Вычисляем дискриминант
            delta = b*b - 4*a*c

            if delta >= 0:
                # Вычисляем корни
                sqrt_delta = int(math.sqrt(delta))
                if sqrt_delta * sqrt_delta == delta:
                    p = (-b + sqrt_delta) // (2*a)
                    q = (-b - sqrt_delta) // (2*a)

                    if p * q == n:
                        return d

    return None

# Пример использования атаки
def demo_wiener_attack():
    # Создаем уязвимую пару ключей RSA
    p = generate_prime(512)
    q = generate_prime(512)
    n = p * q
    phi = (p - 1) * (q - 1)

    # Выбираем малую закрытую экспоненту d
    d = random.randint(1, 10000)

    # Вычисляем открытую экспоненту e
    e = mod_inverse(d, phi)

    print(f"Оригинальные значения:")
    print(f"p = {p}")
    print(f"q = {q}")
    print(f"n = {n}")
    print(f"e = {e}")
    print(f"d = {d}")

    # Применяем атаку
    recovered_d = attack_wiener(e, n)

    print(f"\nРезультат атаки:")
    print(f"Восстановленное значение d = {recovered_d}")
    print(f"Атака успешна: {recovered_d == d}")


In [ ]:
# Упрощенный пример для демонстрации атаки
def demo_simple_wiener_attack():
    # Используем маленькие простые числа
    p = 73
    q = 41
    n = p * q  # n = 2993
    phi = (p - 1) * (q - 1)  # phi = 2880

    # Маленькая закрытая экспонента
    d = 5

    # Вычисляем открытую экспоненту
    e = mod_inverse(d, phi)  # e = 1733

    print(f"Оригинальные значения:")
    print(f"p = {p}")
    print(f"q = {q}")
    print(f"n = {n}")
    print(f"phi = {phi}")
    print(f"e = {e}")
    print(f"d = {d}")

    # Шифруем сообщение
    message = "Test"
    public_key = (e, n)
    cipher = encrypt(message, public_key)
    print(f"Зашифрованное сообщение: {cipher}")

    # Применяем атаку
    recovered_d = attack_wiener(e, n)

    print(f"\nРезультат атаки:")
    print(f"Восстановленное значение d = {recovered_d}")
    print(f"Атака успешна: {recovered_d == d}")

    # Расшифровываем с восстановленным ключом
    if recovered_d:
        private_key = (recovered_d, n)
        decrypted = decrypt(cipher, private_key)
        print(f"Расшифрованное сообщение: {decrypted}")
